In [1]:
##Exercise NN Network using torch.nn module
## Regression
## Reference: CoderzColumn Tutorial

##Load Data
##Normalize Data
##Define Neural Network Model
##Train Model
##Make Predictions
##Evaluate Performance of Model
##Train Model in Batches
##Make Predictions in Batches
##Evaluate Performance of Model

import torch #root package
import pandas as pd
import numpy as np

from torch.utils.data import Dataset, DataLoader #dataset representation and loading

print("PyTorch Version : {}".format(torch.__version__))

PyTorch Version : 2.3.0+cu118


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

print("Device : {}".format(device))

Device : cpu


In [3]:
##Load Data
## Use Boston Data Set

from sklearn import datasets
from sklearn.model_selection import train_test_split

#X and Y are Numpy arrays

##Alternative way to load Boston Housing dataset
from sklearn.datasets import fetch_openml
# Fetch the Boston housing dataset
boston = fetch_openml(name='boston', version=1, parser='auto')

X, Y = boston.data, boston.target

print("X Data")
print(X)
#print(X.dtype)
print(X.values)

print("Y Target")
print(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=123)

##Use float32
tensorX_train = np.array(X_train.values, dtype=np.float32)
tensorX_test = np.array(X_test.values, dtype=np.float32)
tensorY_train = np.array(Y_train.values, dtype=np.float32)
tensorY_test = np.array(Y_test.values, dtype=np.float32)

print(tensorX_train.dtype)

X_train, X_test, Y_train, Y_test = torch.tensor(tensorX_train, dtype = torch.float32),torch.tensor(tensorX_test, dtype=torch.float32),torch.tensor(tensorY_train, dtype=torch.float32),torch.tensor(tensorY_test, dtype=torch.float32)


samples, features = X_train.shape

X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

X Data
        CRIM    ZN  INDUS CHAS    NOX     RM   AGE     DIS RAD    TAX  \
0    0.00632  18.0   2.31    0  0.538  6.575  65.2  4.0900   1  296.0   
1    0.02731   0.0   7.07    0  0.469  6.421  78.9  4.9671   2  242.0   
2    0.02729   0.0   7.07    0  0.469  7.185  61.1  4.9671   2  242.0   
3    0.03237   0.0   2.18    0  0.458  6.998  45.8  6.0622   3  222.0   
4    0.06905   0.0   2.18    0  0.458  7.147  54.2  6.0622   3  222.0   
..       ...   ...    ...  ...    ...    ...   ...     ...  ..    ...   
501  0.06263   0.0  11.93    0  0.573  6.593  69.1  2.4786   1  273.0   
502  0.04527   0.0  11.93    0  0.573  6.120  76.7  2.2875   1  273.0   
503  0.06076   0.0  11.93    0  0.573  6.976  91.0  2.1675   1  273.0   
504  0.10959   0.0  11.93    0  0.573  6.794  89.3  2.3889   1  273.0   
505  0.04741   0.0  11.93    0  0.573  6.030  80.8  2.5050   1  273.0   

     PTRATIO       B  LSTAT  
0       15.3  396.90   4.98  
1       17.8  396.90   9.14  
2       17.8  392.83   4.0

(torch.Size([404, 13]),
 torch.Size([102, 13]),
 torch.Size([404]),
 torch.Size([102]))

In [4]:
samples, features

(404, 13)

In [5]:
## Normalize Data
## To bring the features in same range

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean)/ std
X_test = (X_test - mean)/ std

In [6]:
from torch import nn

class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.first_layer = nn.Linear(features, 5)
        self.second_layer = nn.Linear(5, 10)
        self.third_layer = nn.Linear(10, 15)
        self.final_layer = nn.Linear(15,1)
        self.relu = nn.ReLU()
        ##ReLu - rectified linear units as an activation function for hidden layers

    def forward(self, X_batch):
        layer_out = self.relu(self.first_layer(X_batch))
        layer_out = self.relu(self.second_layer(layer_out))
        layer_out = self.relu(self.third_layer(layer_out))

        return self.final_layer(layer_out)

regressor = Regressor()

preds = regressor(X_train[:5])

preds

tensor([[-0.0463],
        [-0.0191],
        [-0.0586],
        [-0.0456],
        [-0.0510]], grad_fn=<AddmmBackward0>)

In [7]:
## Train NN
def TrainModel(model, loss_func, optimizer, X, Y, epochs=500):
    for i in range(epochs):
        preds = model(X) ## Make Predictions by forward pass through network

        loss = loss_func(preds.ravel(), Y) ## Calculate Loss

        optimizer.zero_grad() ## Zero weights before calculating gradients
        loss.backward() ## Calculate Gradients
        optimizer.step() ## Update Weights

        if i % 100 == 0: ## Print MSE every 100 epochs
            print("MSE : {:.2f}".format(loss))

In [8]:
from torch.optim import SGD, RMSprop, Adam

torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 1000
learning_rate = torch.tensor(1/1e3) # 0.001

regressor = Regressor()
mse_loss = nn.MSELoss()
optimizer = SGD(params=regressor.parameters(), lr=learning_rate)

TrainModel(regressor, mse_loss, optimizer, X_train, Y_train, epochs=epochs)

MSE : 577.23
MSE : 18.63
MSE : 12.35
MSE : 11.01
MSE : 10.39
MSE : 9.97
MSE : 9.42
MSE : 8.81
MSE : 8.44
MSE : 8.22


In [9]:
##Make Predictions

test_preds = regressor(X_test) ## Make Predictions on test dataset

test_preds[:5]

tensor([[20.7536],
        [26.3534],
        [43.9980],
        [22.3184],
        [29.1357]], grad_fn=<SliceBackward0>)

In [10]:
train_preds = regressor(X_train) ## Make Predictions on train dataset

train_preds[:5]

tensor([[47.3433],
        [12.5264],
        [21.4227],
        [27.4200],
        [15.5768]], grad_fn=<SliceBackward0>)

In [11]:
##Evaluate Performance of the Model

from sklearn.metrics import r2_score

print("Train R^2 Score : {:.2f}".format(r2_score(train_preds.detach().numpy().squeeze(), Y_train.detach().numpy())))
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds.detach().numpy().squeeze(), Y_test.detach().numpy())))

Train R^2 Score : 0.89
Test  R^2 Score : 0.71


In [12]:
##Train Model in Batches

def TrainModelInBatches(model, loss_func, optimizer, X, Y, batch_size=32, epochs=500):
    for i in range(epochs):
        batches = torch.arange((X.shape[0]//batch_size)+1) ### Batch Indices

        losses = [] ## Record loss of each batch
        for batch in batches:
            if batch != batches[-1]:
                start, end = int(batch*batch_size), int(batch*batch_size+batch_size)
            else:
                start, end = int(batch*batch_size), None

            X_batch, Y_batch = X[start:end], Y[start:end] ## Single batch of data

            preds = model(X_batch) ## Make Predictions by forward pass through network

            loss = loss_func(preds.ravel(), Y_batch) ## Calculate Loss
            losses.append(loss) ## Record Loss

            optimizer.zero_grad() ## Zero weights before calculating gradients
            loss.backward() ## Calculate Gradients
            optimizer.step() ## Update Weights

        if i % 100 == 0: ## Print MSE every 100 epochs
            print("MSE : {:.2f}".format(torch.tensor(losses).mean()))
            

In [13]:
from torch.optim import SGD, RMSprop, Adam

torch.manual_seed(42) ##For reproducibility.This will make sure that same random weights are initialized each time.

epochs = 1000
learning_rate = torch.tensor(1/1e3) # 0.001
batch_size=32

regressor = Regressor()
mse_loss = nn.MSELoss()
optimizer = SGD(params=regressor.parameters(), lr=learning_rate)

TrainModelInBatches(regressor, mse_loss, optimizer, X_train, Y_train, batch_size=batch_size, epochs=epochs)

MSE : 563.59
MSE : 8.18
MSE : 7.66
MSE : 7.29
MSE : 6.95
MSE : 6.58
MSE : 6.12
MSE : 5.55
MSE : 5.08
MSE : 4.82


In [14]:
##Make Predictions in Batches

def MakePredictions(model, input_data, batch_size=32):
    batches = torch.arange((input_data.shape[0]//batch_size)+1) ### Batch Indices

    with torch.no_grad(): ## Disables automatic gradients calculations
        preds = []
        for batch in batches:
            if batch != batches[-1]:
                start, end = int(batch*batch_size), int(batch*batch_size+batch_size)
            else:
                start, end = int(batch*batch_size), None

            X_batch = input_data[start:end]

            preds.append(model(X_batch))

    return preds
    

In [15]:
test_preds = MakePredictions(regressor, X_test) ## Make Predictions on test dataset

test_preds = torch.cat(test_preds).ravel() ## Combine predictions of all batches

train_preds = MakePredictions(regressor, X_train) ## Make Predictions on train dataset

train_preds = torch.cat(train_preds).ravel()  ## Combine predictions of all batches

In [16]:
##Evaluate Performance of the Model

from sklearn.metrics import r2_score

print("Train R^2 Score : {:.2f}".format(r2_score(train_preds.detach().numpy(), Y_train.detach().numpy())))
print("Test  R^2 Score : {:.2f}".format(r2_score(test_preds.detach().numpy(), Y_test.detach().numpy())))

Train R^2 Score : 0.94
Test  R^2 Score : 0.76
